# Robot Painter

In [17]:
import os

from pydrake.common import temp_directory

import numpy as np
from pydrake.all import (
    AbstractValue,
    ConstantVectorSource,
    DiagramBuilder,
    LeafSystem,
    PiecewisePose,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_robot_painter import TestRobotPainter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddIiwaDifferentialIK
from manipulation.station import MakeHardwareStation, load_scenario

In [23]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [48]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create a temporary file (table_top.sdf)
# in the `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Create a table top SDFormat model.
table_top_sdf_file = os.path.join(temp_dir, "table_top.sdf")
table_top_sdf = """<?xml version="1.0"?>
<sdf version="1.7">

  <model name="table_top">
    <link name="table_top_link">
      <inertial>
        <mass>18.70</mass>
        <inertia>
          <ixx>0.79</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.53</iyy>
          <iyz>0</iyz>
          <izz>1.2</izz>
        </inertia>
      </inertial>
    <visual name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <material>
          <diffuse>0.9 0.9 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1.0e6</drake:hydroelastic_modulus>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="table_top_center">
      <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(table_top_sdf_file, "w") as f:
    f.write(table_top_sdf)

cwd = os.getcwd()
your_model_filename = cwd+"/Board.sdf" # Write the absolute path to your file here

def convert_to_coords(pos):
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return "" + str(.6*.125*(y-3.5)) + ", "+ str(.6*.125*(x-3.5)) 

def get_starting_pose(pos):
    # They are flipped buy oh well
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return RigidTransform([.6*.125*(y-3.5), .6*.125*(x-3.5), 0])

def add_piece(piece_id, name,  pos):
    return f"""
- add_model:
    name: {piece_id}
    file: file://{cwd}/{name}.sdf
    default_free_body_pose:
        {name}:
            translation: [{convert_to_coords(pos)}, .75]
            rotation: !Rpy {{ deg: [0, 0, 0] }}"""

scenario = None
def create_scene():
    global scenario
    scenario_data = f"""
directives:
- add_model:
    name: table_top
    file: file://{table_top_sdf_file}
- add_weld:
    parent: world
    child: table_top::table_top_center
"""
    if your_model_filename:
        scenario_data += f"""
- add_model:
    name: board
    file: file://{your_model_filename}
    default_free_body_pose:
        Board:
            translation: [0, 0, .5]
            rotation: !Rpy {{ deg: [90, 0, 0] }}    
"""
    # scenario_data += add_piece('p1', 'BlackKnight',  'b8')
    # scenario_data += add_piece('p2', 'BlackBishop',  'c8')
    # scenario_data += add_piece('p3', 'BlackKnight',  'g8')
    # scenario_data += add_piece('p4', 'BlackBishop',  'f8')
    # scenario_data += add_piece('p5', 'BlackQueen',  'd8')
    # scenario_data += add_piece('p6', 'BlackRook',  'a8')
    # scenario_data += add_piece('p7', 'BlackRook',  'h8')
    # scenario_data += add_piece('p8', 'BlackKing',  'e8') 
    # scenario_data += add_piece('p9', 'WhiteKnight',  'b1')
    # scenario_data += add_piece('p10', 'WhiteBishop',  'c1')
    # scenario_data += add_piece('p11', 'WhiteKnight',  'g1')
    # scenario_data += add_piece('p12', 'WhiteBishop',  'f1')
    # scenario_data += add_piece('p13', 'WhiteQueen',  'd1')
    # scenario_data += add_piece('p14', 'WhiteRook',  'a1')
    # scenario_data += add_piece('p15', 'WhiteRook',  'h1')
    # scenario_data += add_piece('p16', 'WhiteKing',  'e1')
    # for i in range(8): 
    #    scenario_data += add_piece(f"p{17+i}", 'WhitePawn',  f"{chr(97+i)}2")
    #    scenario_data += add_piece(f"p{25+i}", 'BlackPawn',  f"{chr(97+i)}7")
    
    scenario_data += """
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0, -.55, 0]

- add_model:
    name: gripper
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf
- add_weld:
    parent: iiwa_link_7
    child: gripper::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 0] }
- add_model:
    name: camera
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: gripper::body
    child: camera::base
    X_PC:
        translation: [0, 0.03, 0.04]
        # Point slightly down towards camera
        # RollPitchYaw(0, -0.2, 0.2) @ RollPitchYaw(-np.pi/2, 0, np.pi/2)
        rotation: !Rpy { deg: [-90, 0, 0] }
model_drivers:
    iiwa: !IiwaDriver
        hand_model_name: gripper
    gripper: !SchunkWsgDriver {}
cameras:
    main_camera:
        name: camera0
        depth: True
        X_PB:
            base_frame: camera::base
"""
    with open('board.yaml', 'w') as f:
        f.write(scenario_data)
        
    scenario = load_scenario(data=scenario_data)
    return scenario

In the following cell we provide a wrapper class that hides parts of the implementation details in Drake. You are not required to understand how it works.

In [49]:
class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPose
        )

    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))


class IIWA_Painter:
    def __init__(self, traj=None):
        builder = DiagramBuilder()
        scenario = create_scene()
        self.station = builder.AddSystem(
            MakeHardwareStation(scenario, meshcat=meshcat)
        )
        self.plant = self.station.GetSubsystemByName("plant")
        # Remove joint limits from the wrist joint.
        self.plant.GetJointByName("iiwa_joint_7").set_position_limits(
            [-np.inf], [np.inf]
        )
        controller_plant = self.station.GetSubsystemByName(
            "iiwa.controller"
        ).get_multibody_plant_for_control()

        # optionally add trajectory source
        if traj is not None:
            traj_source = builder.AddSystem(PoseTrajectorySource(traj))
            self.controller = AddIiwaDifferentialIK(
                builder,
                controller_plant,
                frame=controller_plant.GetFrameByName("body"),
            )
            builder.Connect(
                traj_source.get_output_port(),
                self.controller.get_input_port(0),
            )
            builder.Connect(
                self.station.GetOutputPort("iiwa.state_estimated"),
                self.controller.GetInputPort("robot_state"),
            )

            builder.Connect(
                self.controller.get_output_port(),
                self.station.GetInputPort("iiwa.position"),
            )

        gripper_position = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(
            gripper_position.get_output_port(),
            self.station.GetInputPort("gripper.position"),
        )

        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName("body")
        self.world_frame = self.plant.world_frame()

        context = self.CreateDefaultContext()
        self.diagram.ForcedPublish(context)

    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies

        Input:
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.

        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )

    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context
        )

        # provide initial states
        q0 = np.array(
            [
                1.40666193e-05,
                1.56461165e-01,
                -3.82761069e-05,
                -1.32296976e00,
                -6.29097287e-06,
                1.61181157e00,
                -2.66900985e-05,
            ]
        )
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        gripper = self.plant.GetModelInstanceByName("gripper")
        self.plant.SetPositions(plant_context, gripper, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, gripper, [0, 0])

        return context

    def get_X_WG(self, context=None):
        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
            plant_context, frame_A=self.world_frame, frame_B=self.gripper_frame
        )
        return X_WG

    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)

        meshcat.StartRecording(set_visualizations_while_recording=False)
        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)
        meshcat.PublishRecording()

In [69]:
# define center and radius
radius = 0.1
p0 = [0.0, 0.0, 0.0]
R0 = RotationMatrix(np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]]).T)
#R0 = RotationMatrix.MakeXRotation(np.pi/3)
X_WCenter = RigidTransform(R0, p0)

num_key_frames = 10
"""
you may use different thetas as long as your trajectory starts
from the Start Frame above and your rotation is positive
in the world frame about +z axis
thetas = np.linspace(0, 2*np.pi, num_key_frames)
"""
thetas = np.linspace(0, 2 * np.pi, num_key_frames)

painter = IIWA_Painter()

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x5573c001b4b0): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 272
vtkPNGReader (0x5573c001b4b0): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x5573c001b4b0): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 396
vtkPNGReader (0x5573c001b4b0): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x5573c6050870): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 272
vtkPNGReader (0x5573c6050870): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x5573c6050870): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 396
vtkPNGReader (0x5573c6050870): Invalid file header: not a PNG file



In [70]:
X_WG = painter.get_X_WG()
painter.visualize_frame("gripper_current", X_WG)

In [71]:
RotationMatrix.MakeYRotation(np.pi / 6.0)

RotationMatrix([
  [0.8660254037844387, 0.0, 0.49999999999999994],
  [0.0, 1.0, 0.0],
  [-0.49999999999999994, 0.0, 0.8660254037844387],
])

In [72]:
def compose_circular_key_frames(thetas, X_WCenter, X_WGinit):
    """
    returns: a list of RigidTransforms
    """
    # this is an template, replace your code below
    key_frame_poses_in_world = [X_WGinit]
    for theta in thetas:
        # translation is z=cos(theta), x=sin(theta), minus to move counterclockwise
        p_WorldGripper_i = [np.sin(-theta)*radius, 0.0, np.cos(-theta)*radius]
        R_WorldGripper_i = RotationMatrix.MakeYRotation(-theta+np.pi)
        X_GI = RigidTransform(R_WorldGripper_i, p_WorldGripper_i)
        this_pose = X_WCenter @ X_GI
        key_frame_poses_in_world.append(this_pose)

    return key_frame_poses_in_world

In [73]:
# check key frames instead of interpolated trajectory
def visualize_key_frames(frame_poses):
    for i, pose in enumerate(frame_poses):
        painter.visualize_frame("frame_{}".format(i), pose, length=0.05)


key_frame_poses = compose_circular_key_frames(
    thetas, X_WCenter, painter.get_X_WG()
)
visualize_key_frames(key_frame_poses)

In [74]:
X_WGinit = painter.get_X_WG()
total_time = 20
key_frame_poses = compose_circular_key_frames(thetas, X_WCenter, X_WGinit)
times = np.linspace(0, total_time, num_key_frames + 1)
traj = PiecewisePose.MakeLinear(times, key_frame_poses)

Now you should be able to visualize the execution of the circular painting. Use it to confirm that the gripper moves counterclockwise following the key frames previously drawn in the scene.

In [75]:
painter = IIWA_Painter(traj)
painter.paint(sim_duration=total_time)

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x5573cfdb4ff0): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 272
vtkPNGReader (0x5573cfdb4ff0): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x5573cfdb4ff0): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 396
vtkPNGReader (0x5573cfdb4ff0): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x5573cfbfa8c0): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 272
vtkPNGReader (0x5573cfbfa8c0): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x5573cfbfa8c0): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 396
vtkPNGReader (0x5573cfbfa8c0): Invalid file header: not a PNG file



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ea464389-796c-4972-bf7d-0899a1efa759' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>